# Training Intertidal Classification of Europe: Categorising Reflectance of Emerged Areas of Marine vegetation using Sentinel 2

# ICE CREAMS

Import fastbook module and required elements within it to run a tabular learner.

In [1]:
from fastai.tabular.all import *
import glob
import pandas as pd

Read in training data in wide format.
Each version of the model is stored in a separate folder inside Data/Input/Training of the repository. csv files have also been separated into 100,000 row files to minimise file sizes. Therefore, we need to combine them all to create the full training dataframe. 

Training data for ICE CREAMS has 26 features and one label/y column with 9 classes. Features are 12 Bottom of Atmosphere reflectance columns, 12 Min-Max standardised reflectance columns, and 2 columns for Normalised Difference Vegetation Index (NDVI) and Normalised Difference Water Index (NDWI).

In [2]:
Training_Location_V1_1=glob.glob("Data/Input/Training/V1_1/*.csv")
Training_Location_V1_2=glob.glob("Data/Input/Training/V1_2/*.csv")
Training_Location_V1_3=glob.glob("Data/Input/Training/V1_3/*.csv")

In [3]:
Training_df_V1_1 = pd.concat((pd.read_csv(f) for f in Training_Location_V1_1), ignore_index=True)
# Training_df_V1_2 = pd.concat((pd.read_csv(f) for f in Training_Location_V1_2), ignore_index=True)
# Training_df_V1_3 = pd.concat((pd.read_csv(f) for f in Training_Location_V1_3), ignore_index=True)

In [4]:
# df_nn = pd.concat([Training_df_V1_1,Training_df_V1_2,Training_df_V1_3])
df_nn = pd.concat([Training_df_V1_1])

In [5]:
df_nn

,True_Class,Reflectance_B02,Reflectance_B03,Reflectance_B04,Reflectance_B08,Reflectance_B05,Reflectance_B06,Reflectance_B07,Reflectance_B11,Reflectance_B12,...,Reflectance_Stan_B05,Reflectance_Stan_B06,Reflectance_Stan_B07,Reflectance_Stan_B11,Reflectance_Stan_B12,Reflectance_Stan_B8A,Reflectance_Stan_B01,Reflectance_Stan_B09,NDVI,NDWI
0,Chlorophyta,203,350,166,995,691,1091,1112,127,107,...,0.581095,0.979104,1.000000,0.019900,0.0,0.837811,0.234826,0.368159,0.714040,-0.479554
1,Chlorophyta,204,348,176,1058,608,1010,1027,114,98,...,0.531250,0.950000,0.967708,0.016667,0.0,0.812500,0.218750,0.246875,0.714749,-0.504979
2,Chlorophyta,212,335,183,932,608,1010,1027,114,98,...,0.548977,0.981701,1.000000,0.017223,0.0,0.839612,0.226050,0.255113,0.671749,-0.471192
3,Chlorophyta,218,352,177,642,536,692,729,108,88,...,0.698908,0.942278,1.000000,0.031201,0.0,0.680187,0.343214,0.385335,0.567766,-0.291751
4,Chlorophyta,204,340,183,518,536,692,729,108,88,...,0.698908,0.942278,1.000000,0.031201,0.0,0.680187,0.343214,0.385335,0.477889,-0.207459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371120,Bare Sand,833,1136,1170,1386,1308,1361,1375,777,425,...,0.894630,0.948328,0.962513,0.356636,0.0,1.000000,0.247214,0.453901,0.084507,-0.099128
371121,Bare Sand,809,1110,1164,1382,1308,1361,1375,777,425,...,0.894630,0.948328,0.962513,0.356636,0.0,1.000000,0.247214,0.453901,0.085625,-0.109149
371122,Bare Sand,785,1108,1158,1366,1292,1348,1358,694,375,...,0.911531,0.967197,0.977137,0.317097,0.0,1.000000,0.292247,0.495030,0.082409,-0.104285
371123,Bare Sand,795,1100,1132,1360,1292,1348,1358,694,375,...,0.911531,0.967197,0.977137,0.317097,0.0,1.000000,0.292247,0.495030,0.091493,-0.105691


Our labelled column is called True_Class. 

In [6]:
dep_var = "True_Class"
splits = RandomSplitter()(df_nn)

Select which columns are categoric and which are continuous, ICE CREAMS uses just continuous features.

In [7]:
cont_nn,cat_nn = cont_cat_split(df_nn, dep_var=dep_var)

Minimal pre processing is carried out but if any missing data is present it should be filled. 

In [8]:
procs_nn = [FillMissing]
to_nn = TabularPandas(df_nn, procs_nn, cat_nn, cont_nn,
                      splits=splits, y_names=dep_var, y_block=CategoryBlock())

In [9]:
dls = to_nn.dataloaders()

In [10]:
y = to_nn.train.y

Cleate the learner, which is a tabular learner with 9 output classes.

In [11]:
learn = tabular_learner(dls,n_out=9)

Fine tune the learner/model over 20 epochs. Default fine tuning is carried out with ADAM optimiser.

In [12]:
learn.fine_tune(20)

epoch,train_loss,valid_loss,time
0,0.134760,0.120498,00:25


epoch,train_loss,valid_loss,time
0,0.107490,0.099459,00:23
1,0.099683,0.097793,00:26
2,0.096988,0.099995,00:26
3,0.113428,0.097864,00:26
4,0.108169,0.103448,00:25
5,0.106986,0.097521,00:24
6,0.101747,0.099369,00:25
7,0.096095,0.095783,00:23
8,0.095945,0.096034,00:23
9,0.109819,0.097191,00:25


Export the Pickled Model.

In [15]:
learn.export('Models/ICECREAMS_V1_1.pkl')